### moduleのimport

In [ ]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pydicom
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### dicomのパスからimageにする関数を作成

In [ ]:
def dcm_to_image(path):
    d = pydicom.read_file(path)
    dicom = d.pixel_array
    return dicom

### pathを何度も書くのが面倒なのでroot_pathを設定

In [ ]:
root_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

### csvデータをもとにdataframeに

In [ ]:
train = pd.read_csv(root_path + 'train_labels.csv')
test = pd.read_csv(root_path + 'sample_submission.csv')
submission = pd.read_csv(root_path + 'sample_submission.csv')

In [ ]:
print('===========================')
print('train.head() is below')
print('                           ')
print(train.head())
print('                           ')
print('===========================')
print('test.head() is below')
print('                           ')
print(test.head())
print('                           ')
print('===========================')
print('submission.head() is below')
print('                           ')
print(submission.head())
print('                           ')
print('===========================')

## 1.画像の準備

### 1.1.フォルダの中から情報量の多い画像のパスを返す関数の作成

In [ ]:
def dir_path_to_good_image_array(dirpath):
    good_image_path = 0
    num_sum = 0
    for f in glob.glob(dirpath + '*.dcm'):
        d = pydicom.read_file(f)
        data = d.pixel_array
        if num_sum <= np.sum(data):
            num_sum = np.sum(data)
            good_image_path = f

#     print(good_image_path)
    g = pydicom.read_file(good_image_path)
    good_image_array = g.pixel_array
    return good_image_path

# good_image = dir_path_to_good_image_array(root_path + 'train/00000/FLAIR/')
# plt.imshow(good_image)

### 1.2.複数画像での分析のやり方がわからないのでとりあえずFLAIRのgood画像だけで分析してみるので前処理としてIDを変更

In [ ]:
train = pd.read_csv(root_path + 'train_labels.csv')
train["BraTS21ID"] = train["BraTS21ID"].apply(lambda x: str(x).zfill(5))

test = pd.read_csv(root_path + 'sample_submission.csv')
test["BraTS21ID"] = test["BraTS21ID"].apply(lambda x: str(x).zfill(5))

### 1.3.データフレームに分析用のデータ(path)を入れていく

In [ ]:
train['FLAIR'] = 0
a=0
for i in tqdm(train['BraTS21ID']):
    dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/FLAIR/'.format(i)
    train['FLAIR'][a] = dir_path_to_good_image_array(dir_path)
#     dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1w/'.format(i)
#     train['T1w'][a] = dir_path_to_good_image_array(dir_path)
#     dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T1wCE/'.format(i)
#     train['T1wCE'][a] = dir_path_to_good_image_array(dir_path)
#     dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{}/T2w/'.format(i)
#     train['T2w'][a] = dir_path_to_good_image_array(dir_path)
    a+=1

### 1.4.テストにも入れていく

In [ ]:
test['FLAIR'] = 0
a = 0
for i in tqdm(test['BraTS21ID']):
    dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/FLAIR/'.format(i)
    test['FLAIR'][a] = dir_path_to_good_image_array(dir_path)
#     dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/T1w/'.format(i)
#     test['T1w'][a] = dir_path_to_good_image_array(dir_path)
#     dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/T1wCE/'.format(i)
#     test['T1wCE'][a] = dir_path_to_good_image_array(dir_path)
#     dir_path ='../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{}/T2w/'.format(i)
#     test['T2w'][a] = dir_path_to_good_image_array(dir_path)
    a+=1

### 1.5.それぞれの形を軽く見てみる

In [ ]:
print('===========================')
print('train.head() is below\n')
print(train.head())
print('===========================')
print('test.head() is below\n')
print(test.head())
print('===========================')
print('submission.head() is below\n')
print(submission.head())
print('===========================')

### 192 * 192への画像リサイズの関数を作成

In [ ]:
def convert_to_192by192_image(image):
    center = [0, 0]
    center[0] = image.shape[0] // 2
    center[1] = image.shape[1] // 2
    image = image[center[0] - 96 : center[0] + 96, center[1] - 96 : center[1] + 96]
    return image

### 1.6. 訓練用のデータセットに整形したデータを入れていく

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in tqdm(range(len(train))):
    image = dcm_to_image(train['FLAIR'][i])
    image = convert_to_192by192_image(image)
    X_train.append(image)
    y_train.append(train['MGMT_value'][i])

for i in tqdm(range(len(test))):
    image = dcm_to_image(test['FLAIR'][i])
    image = convert_to_192by192_image(image)
    X_test.append(image)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

## 2.ニューラルネットワークの構築

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (192, 192)),
    tf.keras.layers.Dense(64, activation = "relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation = "softmax"),
])

## モデルの学習

In [ ]:
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

model.fit(X_train, y_train, epochs = 120)

## モデルの評価

In [ ]:
model.predict(X_train)

In [ ]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (192, 192)),
    tf.keras.layers.Dense(1000, activation = "relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(500, activation = "relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation = "softmax"),
])

model1.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

"""訓練の過程を表示する"""

history = model1.fit(X_train, y_train, epochs=20)
plt.plot(history.history['accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
y_pred = model1.predict(X_test)
print(y_pred)

In [ ]:
for i in range(len(y_pred)):
    submission['MGMT_value'][i] = y_pred[i].argmax()

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index = False)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    model1,
    show_shapes=True,
)